In [101]:
import pandas as pd
import numpy as np

# Step 1: Read the Excel file
anketa = pd.read_excel("anketa126828-2024-07-20-2.xlsx")

# Step 2: Add a new column to indicate inclusion (1) or exclusion (0)
anketa['included'] = 1

# Social media platforms
social_media = ["Instagram", "TikTok", "FB", "whatsApp", "LinkedIn", "Twitter", "Youtube", "Snapchat", "Threads", "Pinterest", "other"]

# Function to process the data for perception time
i = 50
k = 0
new_columns = {}
while i < 72:
    values_1 = pd.to_numeric(anketa.iloc[:, i], errors='coerce').fillna(0)
    values_2 = pd.to_numeric(anketa.iloc[:, i + 1], errors='coerce').fillna(0)
    values_1[values_1 < 0] = 0
    values_2[values_2 < 0] = 0
    new_column_name = f"{social_media[k]} perception"
    new_columns[new_column_name] = values_1 * 60 + values_2

    k += 1
    i += 2

new_columns_df = pd.DataFrame(new_columns)
df = pd.concat([anketa, new_columns_df], axis=1)

# Function to process the data for actual time
i = 73
k = 0
new_columns = {}
social_media = ["time spend on all social media apps"] + social_media
while i < 98:
    values_3 = pd.to_numeric(df.iloc[:, i], errors='coerce').fillna(0)
    values_4 = pd.to_numeric(df.iloc[:, i + 1], errors='coerce').fillna(0)
    values_3[values_3 < 0] = 0
    values_4[values_4 < 0] = 0
    new_column_name = f"{social_media[k]} actual"
    new_columns[new_column_name] = values_3 * 60 + values_4

    k += 1
    i += 2
    if i == 95:
        i += 1

new_columns_df = pd.DataFrame(new_columns)
df = pd.concat([df, new_columns_df], axis=1)

In [102]:
# Drop the specified columns
df.drop(columns=df.columns[3:16].tolist() + df.columns[27:50].tolist() + df.columns[50:73].tolist() + df.columns[73:97].tolist(), inplace=True)

# Rename columns
df.rename(columns={df.columns[13]: "Perception of time on all social media", df.columns[14]: "Other social media"}, inplace=True)

# Convert columns to numeric and handle negative values
df.replace({-1: np.nan, -2: np.nan}, inplace=True)
df["Perception of time on all social media"] = pd.to_numeric(df["Perception of time on all social media"], errors='coerce') * 60

In [103]:
# New criteria: Update the 'included' column based on status and time difference
df['included'] = df.apply(lambda row: 0 if row['status'] != 6 else row['included'], axis=1)

# Parse the datetime columns
df['t_insert'] = pd.to_datetime(df['t_insert'], format='%d.%m.%Y %H:%M:%S')
df['t_edit'] = pd.to_datetime(df['t_edit'], format='%d.%m.%Y %H:%M:%S')

# Calculate the time difference in minutes and update the 'included' column
df['time_diff'] = (df['t_edit'] - df['t_insert']).dt.total_seconds() / 60
df['included'] = df.apply(lambda row: 0 if row['time_diff'] < 5 else row['included'], axis=1)

#just include the subjects between the age 20 and 25, and sex male and female
df['included'] = df.apply(lambda row: 0 if (row['AGE'] < 20 or row['AGE'] > 25) else row['included'], axis=1)
df['included'] = df.apply(lambda row: 0 if (row['SEX'] ==3) else row['included'], axis=1)

# Drop the 'time_diff' column as it is no longer needed
df.drop(columns=['time_diff'], inplace=True)



In [104]:
columns_to_check = [
    'Instagram actual', 'TikTok actual', 'FB actual', 'whatsApp actual', 
    'LinkedIn actual', 'Twitter actual', 'Youtube actual', 'Snapchat actual', 
    'Threads actual', 'Pinterest actual', 'other actual','time spend on all social media apps actual',
]

# Filtering rows where any value in the specified columns is greater than 660
filtered_df = df[df[columns_to_check].gt(660).any(axis=1)]
print(filtered_df)

# Set 'included' to 0 where any value in the specified columns is greater than 660
df.loc[df[columns_to_check].apply(lambda row: any(row > 660), axis=1), 'included'] = 0


       IDkoda  SEX  AGE  Q2  Q3  Q4  Q5  Q9_2   Q6   Q8  ...  TikTok actual  \
12    0908002    2   21   2   2   1   1   2.0  NaN  1.0  ...              0   
52    bmk69W3    1   22   2   2   1   1   NaN  1.0  NaN  ...              1   
72    268sjow    1   21   1   1   1   1   2.0  NaN  1.0  ...              0   
89    IBAG282    2   22   2   2   1   1   1.0  NaN  2.0  ...            260   
117   L27DKZ1    2   21   2   2   1   1   NaN  2.0  NaN  ...            648   
119   ekpr111    2   22   2   2   1   1   NaN  2.0  NaN  ...            180   
174   5ed3d5d    1   23   2   2   1   1   NaN  1.0  NaN  ...              0   
184    N17N8P    2   22   2   1   1   1   NaN  1.0  NaN  ...           1080   
214  thht0791    2   23   2   2   1   1   2.0  NaN  1.0  ...             60   
220   Nejc153    1   23   2   2   1   1   1.0  NaN  2.0  ...              0   
221   1111LMG    2   21   1   2   2   2   NaN  2.0  NaN  ...              0   
223  h04u07go    2   21   2   2   1   1   NaN  1.0  

In [105]:
columns_to_check = [
    'Perception of time on all social media',
       'Other social media',
       'Instagram perception', 'TikTok perception', 'FB perception',
       'whatsApp perception', 'LinkedIn perception', 'Twitter perception',
       'Youtube perception', 'Snapchat perception', 'Threads perception',
       'Pinterest perception', 'other perception',
]

# Filtering rows where any value in the specified columns is greater than 660
filtered_df = df[df[columns_to_check].gt(660).any(axis=1)]
print(filtered_df['included'])


38     1
54     1
71     0
187    1
201    1
235    1
289    1
382    1
402    1
408    1
455    1
540    1
595    1
610    1
638    0
Name: included, dtype: int64


In [106]:
# Save the updated data to an Excel file
df.to_excel("updated_podatki2.xlsx", index=False)
print(df.describe())

              SEX         AGE          Q2          Q3          Q4          Q5  \
count  644.000000  644.000000  644.000000  644.000000  644.000000  644.000000   
mean     1.624224   21.788820    1.687888    1.729814    1.086957    1.231366   
min      1.000000    2.000000   -3.000000   -3.000000   -3.000000   -3.000000   
25%      1.000000   21.000000    2.000000    2.000000    1.000000    1.000000   
50%      2.000000   21.000000    2.000000    2.000000    1.000000    1.000000   
75%      2.000000   22.000000    2.000000    2.000000    1.000000    2.000000   
max      3.000000   59.000000    2.000000    2.000000    2.000000    2.000000   
std      0.497368    2.924472    0.897782    0.887960    0.818840    0.876979   

             Q9_2          Q6          Q8          Q9  ...  TikTok actual  \
count  337.000000  332.000000  337.000000  332.000000  ...     644.000000   
mean     1.367953    1.093373    0.899110    1.394578  ...      71.487578   
min     -3.000000   -3.000000   -3.0000